In [1]:
#Change the environment to Anaconda 3 before opening Jupyter Notebooks
#this code imports some packages that we'll be using
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os

In [2]:
!git clone https://github.com/dhom-wisc/Data-Sets-Information.git

fatal: destination path 'Data-Sets-Information' already exists and is not an empty directory.


In [3]:
df = pd.read_excel('MP_scenarios.xlsx')
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245


In [4]:
#write the code that solves for one instance of the problem:
num_DVs = 4 #indexed by i
num_constraints = 2 #indexed by j
#Now list all of the given inputs...
#Price and Demand Inputs...
FP = [55.48] #Full Price in dollars
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
D = [120,160,194,223] #Demand at various discount rates
WL = [15] #Number of weeks left
#Salvage value and inventory inputs...
SV = [13.87] #Salvage Value/Unit
RI = [1421] #Remaining Inventory
#Additional Calcuations...
UP = [FP[0]*(1-z)for z in DR] #Unit Prices for each of the decision variables at the various discount rates.

#Step #1: Create a concrete model that we can input information into...
model = ConcreteModel()
#Step #2: Define the bounds of the dependent and indepdendent variables:
model.x = Var(range(num_DVs), domain = NonNegativeReals)
#Step #3 Specify the constraints...
model.Constraints = Constraints = ConstraintList()
for i in range(num_DVs):
    model.Constraints.add(expr = sum(D[i]*model.x[i] for i in range(num_DVs)) <= RI[0])
for i in range(num_DVs):
    model.Constraints.add(expr = sum(model.x[i] for i in range(num_DVs)) <= WL[0])
#Step #4: Secify the objective for the linear optimization model
model.Objective = Objective(expr = sum(UP[i]*D[i]*model.x[i] for i in range(num_DVs))+(SV[0]*(RI[0] - sum(D[i]*model.x[i] for i in range(num_DVs)))), sense = maximize)
#Step#5: Specify the solver to solve the optimization problem
opt = SolverFactory('glpk')
results = opt.solve(model)
#Step #6 Print out your results.
print(model.Objective()), print(UP)
#print the optimal decision variables
for i in range(num_DVs):
        print(model.x[i],":", value(model.x[i]) )

78837.08000000016
[55.48, 47.157999999999994, 38.836, 27.74]
x[0] : 11.8416666666667
x[1] : 0.0
x[2] : 0.0
x[3] : 0.0


In [5]:
def mozart(UP,D,WL,SV,RI,num_DVs):
    #Additional Calcuations...
    #Step #1: Create a concrete model that we can input information into...
    model = ConcreteModel()
    #Step #2: Define the bounds of the dependent and indepdendent variables:
    model.x = Var(range(num_DVs), domain = NonNegativeReals)
    #Step #3 Specify the constraints...
    model.Constraints = Constraints = ConstraintList()
    for i in range(num_DVs):
        model.Constraints.add(expr = sum(D[i]*model.x[i] for i in range(num_DVs)) <= RI[0])
    for i in range(num_DVs):
        model.Constraints.add(expr = sum(model.x[i] for i in range(num_DVs)) <= WL[0])
    #Step #4: Secify the objective for the linear optimization model
    model.Objective = Objective(expr = sum(UP[i]*D[i]*model.x[i] for i in range(num_DVs))+(SV[0]*(RI[0] - sum(D[i]*model.x[i] for i in range(num_DVs)))), sense = maximize)
    #Step#5: Specify the solver to solve the optimization problem
    opt = SolverFactory('glpk')
    results = opt.solve(model)
    #Step #6 Print out your results.
    solution = []
    solution.append('obj =' + str(model.Objective()))
    #get some assistance from the professor on this issue...
    solution.append('x1=' + str(model.x[0]()))
    solution.append('x2=' + str(model.x[1]()))
    solution.append('x3=' + str(model.x[2]()))
    solution.append('x4=' + str(model.x[3]()))
    solution.append('Sales Price =' + str(UP[0]))
    #solution.append('Price2=' + str(UP[1]))
    #solution.append('Price3=' + str(UP[2]))
    #solution.append('Price4=' + str(UP[3]))
    return(solution)

In [6]:
#write the code that solves for one instance of the problem:
num_DVs = 4 #indexed by i
num_constraints = 2 #indexed by j
#Now list all of the given inputs...
#Price and Demand Inputs...
FP = [55.48] #Full Price in dollars
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
D = [120,160,194,223] #Demand at various discount rates
WL = [15] #Number of weeks left
#Salvage value and inventory inputs...
SV = [13.87] #Salvage Value/Unit
RI = [1421] #Remaining Inventory
#Additional Calcuations...
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
UP = [FP[0]*(1-z)for z in DR] #Unit Price
mozart(UP,D,WL,SV,RI,num_DVs)

['obj =78837.08000000016',
 'x1=11.8416666666667',
 'x2=0.0',
 'x3=0.0',
 'x4=0.0',
 'Sales Price =55.48']

In [7]:
#write the code that solves for one instance of the problem:
num_DVs = 4 #indexed by i
num_constraints = 2 #indexed by j
#Now list all of the given inputs...
#Price and Demand Inputs...
FP = [55.48] #Full Price in dollars
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
D = [120,160,194,223] #Demand at various discount rates
WL = [15] #Number of weeks left
#Salvage value and inventory inputs...
SV = [13.87] #Salvage Value/Unit
RI = [1421] #Remaining Inventory
#Additional Calcuations...
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
UP = [FP[0]*(1-z)for z in DR] #Unit Price
mozart(UP,D,WL,SV,RI,num_DVs)

['obj =78837.08000000016',
 'x1=11.8416666666667',
 'x2=0.0',
 'x3=0.0',
 'x4=0.0',
 'Sales Price =55.48']

In [8]:
#lets check out one instance of the process to solve for one row of data...
num_DVs = 4 #indexed by i
num_constraints = 2 #indexed by j
k = 0 #which row
rowdata3 = df.iloc[k].values.tolist()
FP = [rowdata3[1]]
D = rowdata3[2:6]
WL = [rowdata3[6]]
SV = [rowdata3[8]]
RI = [rowdata3[7]]
#Additional Calcuations...
DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
UP = [rowdata3[1]*(1-z)for z in DR] #Unit Price
mozart(UP,D,WL,SV,RI,num_DVs)

['obj =78837.08000000016',
 'x1=11.8416666666667',
 'x2=0.0',
 'x3=0.0',
 'x4=0.0',
 'Sales Price =55.48']

In [9]:
#lets check out one instance of the process to solve for one row of data...
num_DVs = 4 #indexed by i
num_constraints = 2 #indexed by j
outputs3 = []
for k in range(len(df)):
    rowdata3 = df.iloc[k].values.tolist()
    rowdata3
    FP = [rowdata3[1]]
    D = rowdata3[2:6]
    WL = [rowdata3[6]]
    SV = [rowdata3[8]]
    RI = [rowdata3[7]]
    DR = [0,0.15,0.30,0.50] #Discount Rates indexed by z
    #Additional Calcuations...
    UP = [rowdata3[1]*(1-z)for z in DR] #Unit Price
    outputs3.append(mozart(UP,D,WL,SV,RI,num_DVs))
outputs3

[['obj =78837.08000000016',
  'x1=11.8416666666667',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =55.48'],
 ['obj =104139.2',
  'x1=0.0',
  'x2=15.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =53.68'],
 ['obj =141254.12752941175',
  'x1=6.29411764705882',
  'x2=8.70588235294118',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =61.56'],
 ['obj =86487.5200000002',
  'x1=11.4434782608696',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =65.72'],
 ['obj =89477.46',
  'x1=11.475',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =64.98'],
 ['obj =61105.0',
  'x1=11.0',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =50.5'],
 ['obj =80712.76000000004',
  'x1=8.58518518518519',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =69.64'],
 ['obj =115386.74387755073',
  'x1=13.469387755102',
  'x2=1.53061224489795',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =62.84'],
 ['obj =81322.00000000012',
  'x1=13.2727272727273',
  'x2=0.0',
  'x3=0.0',
  'x4=0.0',
  'Sales Price =55.7'],
 ['obj =7864

In [10]:
#Let's put this back into the data set
df['solution'] = outputs3
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,solution
0,1,55.48,120,160,194,223,15,1421,13.870,"[obj =78837.08000000016, x1=11.8416666666667, ..."
1,2,53.68,115,149,171,197,15,2396,13.420,"[obj =104139.2, x1=0.0, x2=15.0, x3=0.0, x4=0...."
2,3,61.56,140,191,207,459,15,2544,15.390,"[obj =141254.12752941175, x1=6.29411764705882,..."
3,4,65.72,115,151,242,278,15,1316,16.430,"[obj =86487.5200000002, x1=11.4434782608696, x..."
4,5,64.98,120,173,221,335,15,1377,16.245,"[obj =89477.46, x1=11.475, x2=0.0, x3=0.0, x4=..."


In [11]:
#export to csv
df.to_excel('MP_scenarios_solutions.xlsx', index = 'No')